# School shootings paper

Some bibliography: https://www.mendeley.com/community/school-shootings-bib-(until-2015)

In [10]:
import sys
sys.path.append('./')

import pandas as pd
from numberAttacks import *
from pylab import *
#from cosas import *
from datetime import datetime

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [11]:
def regetDataDaysMiami_v2(fname,state=0,type=0,time=0):
    
    f1 = open(fname)
    #Add data to array
    x = 1991
    z = datetime(x,1,1)
    j = []
    countV = np.zeros(25)
    countV2 = np.zeros(25)
    count = 0
    sizeEvent = []
    sizeEvent2 = []
    for line in f1:
        a =line.split(';')
        dat = datetime.strptime(a[0],"%m %d %Y")
        try:

            if state: condState = a[10] == state
            else: condState = 1

            if type == 'NCollege': condType = a[8] != type[1:]
            elif type: condType = a[8] == type
            else: condType = 1

            if time: condTime = dat > datetime(year=time,month=1,day=1)
            else: condTime = 1
            if condState and condType and condTime:
                countV[dat.year-1990] += int(a[6])
                countV2[dat.year-1990] += 1

                j.append(dat)
                sizeEvent.append(int(a[6]))
                sizeEvent2.append(int(a[6]))

        except ValueError: pass


    sizeEvent = [sizeV for (date, sizeV) in sorted(zip(j,sizeEvent))]
    sizeEvent2 = [sizeV for (date, sizeV) in sorted(zip(j,sizeEvent2))]
    j = sorted(j)
    try: vtm = j.index(datetime.strptime("04 16 2007","%m %d %Y"))
    except ValueError: vtm = 0

    #print x, countV
    ddd = j


    a = [n.total_seconds()/60/60/24 for n in diff(j)]
    daysBetweenKills = np.asarray(a)

    daysBetweenKills[daysBetweenKills==0] += 0.1

    #Separate attacks in the same day
    while(np.any(np.diff(daysBetweenKills)==0)):
        daysBetweenKills[np.concatenate([[False],np.diff(daysBetweenKills)==0])] += 0.1


    daysBetweenAttacks = daysBetweenKills

    return [ddd,np.asarray(sizeEvent),np.asarray(sizeEvent2),np.asarray(daysBetweenAttacks),countV,countV2,vtm]


In [ ]:
def fig1A(fname):
    fname = 'ShootMiami2.csv'
    #fname = 'USAToday.csv'
    fname = "./data/"+fname
    if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname)
    elif fname == "USAToday.csv": [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = studyUSAToday(fname)
    print(sizeEvent)
    [xCCDF,yCCDF,PDF] = get_CCDF(sizeEvent)


    fig = figure(figsize=(3,3))
    ax = fig.add_subplot(1,1,1)
    print("Med", np.median(sizeEvent))

    xmin = findXmin(sizeEvent)

    plotGlobalPL(sizeEvent,xmin=xmin,ax=ax)
    customaxis(ax,size=11,lw=2)
    xlabel('Severity of Attacks (s)',fontsize = 12)
    xlabel(r'$P(X>s)$',fontsize = 12)

    savefig(fname+'PL_Dead_Miami_v2.pdf', bbox_inches='tight' ,dpi=100)
    show()

In [ ]:

def testXx(fname = 'everytown.csv',state=0,type=0,time=0):

    from datetime import datetime
    fig = figure(figsize=(4.5,3))
    ax = fig.add_subplot(1,1,1)
    f1 = open(fname)
    #Add data to array
    x = 1991
    j = []
    pepe = dict()
    n = []
    for line in f1:
        a =line.split(';')
        dat = datetime.strptime(a[1],"%m %d %Y")

        if dat == datetime.strptime("10 01 2007","%m %d %Y"): vtm = len(j)
        try:
            n.append((dat - datetime(2013,1,1)).total_seconds()/60./60./24.)
            city= a[4] #3 state
            t = city
            if t == "CA": print dat

            if pepe.get(t):
                pepe[t].append(dat)
            else:
                pepe[t] = [dat]
        except: pass

    temp = []
    for i in range(len(n)):
        for j in range(i,len(n)):

            temp.append([np.abs(n[i]-n[j])])
    print np.median(temp)


    print pepe
    lab = []
    pt = []
    i = 0
    for city in pepe:
        if  len(pepe[city]) > 1:
            i += 1
            tac = pepe[city]
            tac = [(_ - datetime(2013,1,1)).total_seconds()/60./60./24. for _ in tac]
            print dcity, tac
            pt += [np.median(np.diff(tac))]
            lab.append(city)
            for date in pepe[city]:
                ax.plot(date,i,'.',ms=10,color='red',alpha=0.5)

    customaxis(ax, c_right='none', c_top='none',lw=2, size=10, pad=8)
    plt.yticks(range(1,1+len(lab)),lab)
    plt.ylim((0.5,len(lab)+.5))
    plt.xticks(rotation=30)
    print np.median(pt)
    savefig('CitiesCorr.pdf', bbox_inches='tight' ,dpi=10)
    plt.show()
    print len(pepe)

def testX(fname = 'ShootMiami2.csv',state=0,type=0,time=0):
    from datetime import datetime
    f1 = open(fname)
    #Add data to array
    fig = figure(figsize=(4.5,3))
    ax = fig.add_subplot(1,1,1)
    pepe = dict()
    for line in f1:
        a =line.split(';')
        dat = datetime.strptime(a[0],"%m %d %Y")
        try:
            city= a[10]
            t = city
            print t

            if pepe.get(t):
                pepe[t].append(dat)
            else:
                pepe[t] = [dat]
        except: pass
    print pepe
    lab = []
    i = 0


    for city in ['CA', 'FL', 'GA', 'MI', 'WA','TN', 'PA',   'IL','AL','DC', 'TX',  'NY',  'OH']:
        if  len(pepe[city]) > 5:
            i += 1
            tac = pepe[city]
            tac = [(_ - datetime(1990,1,1)).total_seconds() for _ in tac]
            print np.std(tac)
            lab.append(city)
            for date in pepe[city]:
                ax.plot(date,i,'.',ms=10,color='red',alpha=0.5)

    customaxis(ax, c_right='none', c_top='none',lw=2, size=10, pad=8)
    plt.yticks(range(1,1+len(lab)),lab)
    plt.ylim((0.5,len(lab)+.5))
    plt.xticks(rotation=30)

    savefig('StatsSchultzCorr.pdf', bbox_inches='tight' ,dpi=10)
    print lab


    plt.show()



def studyTweets_v2(time=0):
    import os
    import pylab as plt
    import numpy as np
    from datetime import datetime
    import pandas as pd

    listFiles = os.listdir("/home/j/Documents/Dropbox overflow/Twitter database/datav2/")

    d = dict()

    for fileT in listFiles:
        if ".tgz" in fileT:
            name = fileT[:-10]
            name = "20" + name[-2:] +"-"+ name[3:5] + "-" + name[0:2]
        elif ".gz" in fileT:
            name = fileT[:-15]
        else: name = 0

        if name:
            with open("/home/j/Documents/Dropbox overflow/Twitter database/datav2/"+fileT) as f:
                t = np.asarray([int(_.strip()) for _ in f.readlines()])
                if len(t) > 0:
                    if d.has_key(name):
                        d[name] += t
                    else:
                        d[name] = t


    dates = []
    tweetsAll = []
    tweetsSS = []
    tweetsMS = []
    tweetsMM = []
    tweetsS = []
    for name in d:
        if time:
            if int(name[0:4]) > time:
                dates.append(datetime (year=int(name[0:4]),month=int(name[5:7]),day=int(name[-2:])))

                tweetsAll.append(float(d[name][0]))
                tweetsSS.append(float(d[name][1]))
                tweetsMS.append(float(d[name][2]))
                tweetsMM.append(float(d[name][3]))
                tweetsS.append(float(d[name][4]))
        else:
            dates.append(datetime (year=int(name[0:4]),month=int(name[5:7]),day=int(name[-2:])))

            tweetsAll.append(float(d[name][0]))
            tweetsSS.append(float(d[name][1]))
            tweetsMS.append(float(d[name][2]))
            tweetsMM.append(float(d[name][3]))
            tweetsS.append(float(d[name][4]))


    tweetsAll = np.asarray([tweet for (date,tweet) in sorted(zip(dates,tweetsAll))])
    tweetsSS = np.asarray([tweet for (date,tweet) in sorted(zip(dates,tweetsSS))])
    tweetsMS = np.asarray([tweet for (date,tweet) in sorted(zip(dates,tweetsMS))])
    tweetsMM = np.asarray([tweet for (date,tweet) in sorted(zip(dates,tweetsMM))])
    tweetsS = np.asarray([tweet for (date,tweet) in sorted(zip(dates,tweetsS))])
    datesFinal = sorted(dates)
    print np.sum(tweetsAll)
    print np.sum(tweetsSS)
    print np.sum(tweetsMM)
    print np.sum(tweetsMS)
    print np.sum(tweetsS)
    return [datesFinal,tweetsS,tweetsSS,tweetsAll,tweetsMS,tweetsMM]

def regetDataDaysEveryTown_v2(fname,state=0,type=0,time=0):
    from datetime import datetime
    f1 = open(fname)
    #Add data to array
    x = 1991
    z = datetime(x,1,1)
    j = []
    countV = []
    countV2 = np.zeros(25)
    count = 0
    sizeEvent = []
    sizeEvent2 = []
    vtm = 0
    for line in f1:
        a =line.split(';')
        dat = datetime.strptime(a[1],"%m %d %Y")
        if dat == datetime.strptime("10 01 2007","%m %d %Y"): vtm = len(j)
        try:

            #3 state
            # 6 college
            #7 gun fired
            #8 injured
            #9 death
            #10 attemted suicide
            #11 suicide

            if state: condState = a[3] == state
            else: condState = 1

            if type == 'NCollege': condType = a[6] != type[1:]
            elif type: condType = a[6] == type
            else: condType = 1

            if time: condTime = dat > datetime.strptime("1 1 2009","%m %d %Y")
            else: condTime = 1
            if condState and condType and condTime:

                countV2[dat.year-1990] += 1
                if a[6] =='College':
                    countV.append(0)
                else:
                    countV.append(1)
                j.append(dat)
                sizeEvent.append(int(a[9]))
                sizeEvent2.append(int(a[11]))

        except ValueError: pass

    #print x, countV
    ddd = j


    a = [n.total_seconds()/60/60/24 for n in diff(j)]
    daysBetweenKills = np.asarray(a)

    daysBetweenKills[daysBetweenKills==0] += 0.1

    #Separate attacks in the same day
    while(np.any(np.diff(daysBetweenKills)==0)):
        daysBetweenKills[np.concatenate([[False],np.diff(daysBetweenKills)==0])] += 0.1


    daysBetweenAttacks = daysBetweenKills

    return [ddd,np.asarray(sizeEvent),np.asarray(sizeEvent2),np.asarray(daysBetweenAttacks),countV,countV2,vtm]



def studyUSAToday(fname,state=0,type=0,time=0):
    from datetime import datetime
    f1 = open(fname)
    #Add data to array
    x = 1991
    z = datetime(x,1,1)
    j = []
    countV = []
    countV2 = np.zeros(25)
    count = 0
    sizeEvent = []
    sizeEvent2 = []
    vtm = 0
    for lineNumber, line in enumerate(f1):
        if lineNumber == 0: continue
        a =line.split('\t')
        dat = datetime.strptime(a[0],"%b %d, %Y")
        if dat == datetime.strptime("10 01 2007","%m %d %Y"): vtm = len(j)
        try:
            #3 state
            # 6 college
            #7 gun fired
            #8 injured
            #9 death
            #10 attemted suicide
            #11 suicide
            if state:
                condState = a[2] == state
            else: condState = 1

            #print(a[-2])
            if a[-3] == "Shooting":
                condType = 1
            else:
                condType = 0

            if time: condTime = dat > datetime(year=time,month=1,day=1)
            else: condTime = 1
            if condState and condType and condTime:
                #countV2[dat.year-1990] += 1
                countV.append(1)
                j.append(dat)
                sizeEvent.append(int(a[-1]))
                sizeEvent2.append(int(a[-1]))

        except ValueError: pass

    #print x, countV
    ddd = j


    a = [n.total_seconds()/60/60/24 for n in diff(j)]
    daysBetweenKills = np.asarray(a)

    daysBetweenKills[daysBetweenKills==0] += 0.1

    #Separate attacks in the same day
    while(np.any(np.diff(daysBetweenKills)==0)):
        daysBetweenKills[np.concatenate([[False],np.diff(daysBetweenKills)==0])] += 0.1


    daysBetweenAttacks = daysBetweenKills

    return [ddd,np.asarray(sizeEvent),np.asarray(sizeEvent2),np.asarray(daysBetweenAttacks),countV,countV2,vtm]






def residualsMiami_v2():
    from scipy.stats.stats import pearsonr
    import scipy
    name = ''
    i=1

    for fname in [ "everytown.csv"]:#['ShootMiami2.csv','everytown.csv']:#
        if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname,time=2010)
        elif fname == "USAToday.csv": [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = studyUSAToday(fname,time=2010)
        else: [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2(fname)


        print sizeEvent
        print sizeEvent2
        #sleep()
        print pearsonr(sizeEvent,sizeEvent2)
        x = np.median(sizeEvent)
        print x
        print np.mean(sizeEvent2[sizeEvent<=x]),np.std(sizeEvent2[sizeEvent<=x])
        print np.mean(sizeEvent2[sizeEvent>x]),np.std(sizeEvent2[sizeEvent>x])


        [xCCDF,yCCDF,PDF] = get_CCDF(sizeEvent)
        #print len(sizeEvent)
        dates =[0,len(sizeEvent)-1]
        [_,_,_,_,_,x_p,y_p] = plotFrequency(dates, daysBetweenAttacks,1,1,1,name,vtm,ddd=ddd)

        x_r = np.log10(np.arange(1,len(daysBetweenAttacks)+1))
        y_r = np.log10((daysBetweenAttacks))
        x = y_p-y_r

        clf()
        plt.show()
        fig2 = figure(2,figsize=(4.5,3))
        dist_name = 'norm'
        axX = fig2.add_subplot(1,1,1)

        h = 7
        axX.hist(x,bins=np.linspace(-1.5,1.5,h),normed=1,edgecolor='white',facecolor='gray',alpha=0.5)

        dist = getattr(scipy.stats, dist_name)
        param = dist.fit(x)
        x_pl = np.linspace(-1.5,1.5,100)
        pdf_fitted = dist.pdf(x_pl, *param[:-2], loc=param[-2], scale=param[-1])
        axX.plot(x_pl, pdf_fitted, linewidth=3,alpha=0.5,label='Normal Fit')
        axX.legend()
        axX.set_xlabel(r"$\sum{\epsilon}$")
        axX.set_ylabel("Probability")
        customaxis(axX, c_right='none', c_top='none',lw=2, size=10, pad=8)
        savefig(fname+'DistributionOutliers.pdf', bbox_inches='tight' ,dpi=10)
        plt.show()
        print(np.mean(x),np.std(x))
        indexLate = np.nonzero(x<(np.mean(x)-1.*np.std(x)))[0]+1
        indexEarly = np.nonzero(x>=(np.mean(x)+1.*np.std(x)))[0]+1
        indexEarly = np.array(indexEarly,dtype="int")
        indexLate = np.array(indexLate,dtype="int")
        indLate = len(indexLate)
        indEarly = len(indexEarly)


        fig = figure(1, figsize=(9,6))
        ax1 = fig.add_subplot(2,1,1)
        [datesFinal,tweetsS,tweetsSS,tweetsAll,tweetsMS,tweetsMM] = studyTweets_v2(time=2009)

        tweetsFinal = tweetsSS/tweetsAll
        [datesFinal,tweetsFinal] = zip(*[(date,tweets)  for (date,tweets) in zip(datesFinal,tweetsFinal) if date>ddd[0]])
        ax1.plot(datesFinal,1E6*np.asarray(tweetsFinal),'-',marker='.',linewidth=2,color='red',alpha=0.8)


        ax1.set_ylabel('Number of tweets containing \n\"school shooting\" per million',fontsize=11)
        ax1.set_ylim([0,350])
        ax2 = ax1.twinx()

        tweetsBeforeAttackEarly = np.zeros(indEarly)
        tweetsBeforeAttackLate = np.zeros(indLate)
        tweetsNotBeforeAttack = np.zeros(len(ddd))

        sizeBeforeAttackEarly = np.zeros(indEarly)
        sizeBeforeAttackLate = np.zeros(indLate)
        sizeNotBeforeAttack = np.zeros(len(ddd))

        i = -1
        for dateX in ddd:
            i += 1
            ax2.vlines(dateX,0,sizeEvent[i],color = 'gray',linewidth=2)
            tweetsBefore = [tweets for (date,tweets) in zip(datesFinal,tweetsFinal) if date<dateX][-5:]
            if len(tweetsBefore) == 0:
                tweetsNotBeforeAttack[i] = np.NaN
            else:
                tweetsNotBeforeAttack[i] = np.max(tweetsBefore)
            sizeNotBeforeAttack[i] = sizeEvent2[i]



        i = -1
        print(indexEarly)
        for element in indexEarly:
            #print ddd[element], element

            i += 1
            ax2.vlines(ddd[element],0,sizeEvent[element],color = (73./256, 142./256, 204./256),linewidth=3)
            tweetsBefore = [tweets for (date,tweets) in zip(datesFinal,tweetsFinal) if date<ddd[element]][-5:]

            tweetsBeforeAttackEarly[i] = np.max(tweetsBefore)
            sizeBeforeAttackEarly[i] = sizeEvent2[element]


        print(indexLate)
        i = -1
        for element in indexLate:
            i += 1
            ax2.vlines(ddd[element],0,sizeEvent[element],color = 'orange',linewidth=3)
            tweetsBefore = [tweets for (date,tweets) in zip(datesFinal,tweetsFinal) if date<ddd[element]][-5:]

            tweetsBeforeAttackLate[i] = np.max(tweetsBefore)
            sizeBeforeAttackLate[i] = sizeEvent2[element]

        print "sizes"
        print sizeBeforeAttackLate
        print sizeBeforeAttackEarly


        ax2.set_ylabel('Size of Attack',fontsize=11)
        ax1.set_xlabel('Time',fontsize=11)
        customaxis(ax2, c_right='k', c_top='none',lw=2, size=10, pad=8)
        customaxis(ax1, c_right='k', c_top='none',lw=2, size=10, pad=8)

        ax2.set_ylim([0,4])
        #ax1.set_xlim([datetime.datetime(2012,11,30),datetime.datetime(2013,3,30)])
        #ax2.set_xlim([datetime.datetime(2012,11,30),datetime.datetime(2013,3,30)])
        plt.clf()
        ax5 = fig.add_subplot(2,1,2)
        tweetsNotBeforeAttack *= 1E6
        tweetsBeforeAttackLate *= 1E6
        tweetsBeforeAttackEarly *= 1E6

        print [np.median(tweetsNotBeforeAttack[~np.isnan(tweetsNotBeforeAttack)]),np.median(tweetsBeforeAttackEarly),
                           np.median(tweetsBeforeAttackLate)]
        print [np.mean(sizeNotBeforeAttack[~np.isnan(sizeNotBeforeAttack)]),np.mean(sizeBeforeAttackEarly),np.mean(sizeBeforeAttackLate)]
        print sizeBeforeAttackEarly
        print sizeBeforeAttackLate
        color = ['gray',(73./256, 142./256, 204./256),'orange']
        width = 0.4
        ax5.bar([1,1.5,2],[np.median(tweetsNotBeforeAttack[~np.isnan(tweetsNotBeforeAttack)]),np.median(tweetsBeforeAttackEarly),
                           np.median(tweetsBeforeAttackLate)],width=width,color=color)
                            #yerr = [np.std(tweetsNotBeforeAttack[~np.isnan(tweetsNotBeforeAttack)]),np.std(tweetsBeforeAttackEarly)
                            #    ,np.std(tweetsBeforeAttackLate)], width=width,color=color)
        ax5.set_ylabel('Median number of tweets in the \n5 days previous to the attack')


        ax6 = ax5.twinx()
        color = ['gray',(73./256, 142./256, 204./256),'orange']
        ax6.bar([3,3.5,4],[np.mean(sizeNotBeforeAttack[~np.isnan(sizeNotBeforeAttack)]),np.mean(sizeBeforeAttackEarly),np.mean(sizeBeforeAttackLate)],width=width,color=color)
                #yerr=[np.std(sizeNotBeforeAttack[~np.isnan(sizeNotBeforeAttack)]),np.std(sizeBeforeAttackEarly),np.std(sizeBeforeAttackLate)],width=width,color=color)
        if fname == 'everytown.csv':
            ax6.set_ylabel('Average suicide rate')
        else:
            ax6.set_ylabel('Average casualty number')

        plt.xticks(np.asarray([1,1.5,2,3,3.5,4])+0.2,["All","Early","Late","All","Early","Late"])
        customaxis(ax6, c_right='k', c_top='none',lw=2, size=10, pad=8)
        customaxis(ax5, c_right='k', c_top='none',lw=2, size=10, pad=8)
        savefig(fname+'StimelineOutliers.pdf', bbox_inches='tight' ,dpi=10)
        show()


def fig1BCD():
    for fname in ["USAToday.csv"]:#['ShootMiami2.csv']:
        for name in [0]:# ['NCollege','College',0]:
            if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname,type=name)
            elif fname == "USAToday.csv": [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = studyUSAToday(fname,type=name)
            else: [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2(fname,type=name)
            print(len(ddd))

            if name == 'NCollege': name = 'K12'
            if name == 0: name = 'All'

            [xCCDF,yCCDF,PDF] = get_CCDF(sizeEvent)
            fig = figure(1)

            dates =[0,len(sizeEvent)-1]
            fig = figure(figsize=(3.,3))
            ax1 = fig.add_subplot(1,1,1)

            plotFrequency(dates, daysBetweenAttacks,1,1,1,'Mass',vtm,ax=ax1,ddd=ddd,allX=False)

            #savefig('lowessMass.pdf', bbox_inches='tight' ,dpi=100)
            xlabel(r'Log10 Attack number, $n$')
            ylabel(r'Log10 Time between attacks $\tau_n$')
            title(name)
            plt.gca().set_ylim(bottom=0)
            savefig(str(name)+fname+'temp_Evo_Dead_Miami_v2.pdf', bbox_inches='tight' ,dpi=100)
            show()
            clf()

def fig1EF(returnX = False,date=0):
    for fname in ['ShootMiami2.csv']:#,"USAToday.csv"]:
        for name in [0]:

            if fname == "USAToday.csv": stateList = [  'Ariz.',  'Calif.',  'Fla.',  'Ill.', 'Ind.', 'Kan.', 'La.',   'Md.', 'Mich.',  'Mo.',        'N.C.', 'N.Y.', 'Ohio', 'Okla.', 'S.C.', 'Tenn.', 'Texas','Va.', 'W.Va.', 'Wash.', 'Wis.']
            if fname == 'ShootMiami2.csv': stateList = ["AL","CA","DC","FL","GA","IL","KY","LA","MI","MN","MS","NC","NY","OH","PA","SC","TN","TX","WA"]
            if fname == 'everytown.csv': stateList = ["CA","FL","GA","KY","LA","MI","MN","MS","NC","NY","OH","PA","SC","TN","TX","WA"]
            t2 = len(stateList)+1

            tau1 = np.zeros(t2-1)
            slope = np.zeros(t2-1)
            t = np.ceil(np.sqrt(t2))
            i = 0
            j = -1
            listSlopes =[]
            listX = []
            listStates = []
            axis1 = np.ceil(np.sqrt(len(stateList)))
            axis2 = np.ceil(np.sqrt(len(stateList)))
            for state in stateList:
                if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2('ShootMiami2.csv',state=state,type=name,time=date)
                elif fname == 'everytown.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2('everytown.csv',state=state,type=name)
                elif fname == 'USAToday.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = studyUSAToday("USAToday.csv",state=state)
                j += 1
                i += 1
                print state,len(ddd)
                dates =[0,len(sizeEvent)-1]
                ## To study last 5 attacks
                #daysBetweenAttacks = daysBetweenAttacks[max([-5,-len(daysBetweenAttacks)]):]
                #dates = [0, len(daysBetweenAttacks)]

                ## Normal
                ax = subplot(axis1,axis2,i)
                title(state)
                [tau1[i-1],slope[i-1],listSlopesT,listXT,listStatesT,temp1,temp2] = plotFrequency(dates, daysBetweenAttacks,t,t,i,state,vtm,ax=ax,ddd=ddd,allX=False)

                listSlopes = listSlopes + listSlopesT
                listX = listX + listXT
                listStates = listStates + listStatesT


            i += 1
            subplot(t,t,j)
            hold(True)
            #savefig(name+'K12toptopStates_Miami_AllStates.png', bbox_inches='tight' ,dpi=100)
            #savefig('temp1.png', bbox_inches='tight' ,dpi=100)
            fig = figure(figsize=(6.,3))
            ax = fig.add_subplot(1,1,1)

            print tau1, slope
            #stateList = ["AL","CAD","CAE","DC","FL","GA","IL","KY","LA","MI","MN","MS","NC","NY","PA","SC","TN","TX","WA"]
            for i2 in range(len(tau1)):
                #print st, i2
                ax.plot(tau1[i2],slope[i2],'o')
                ax.annotate(stateList[i2],(tau1[i2],slope[i2]+0.1))



            slope2b, interceptb, r_valueb, p_value, std_err = linregress(tau1[~np.isnan(slope)],slope[~np.isnan(slope)])
            print("MMMM",std_err)
            if returnX == True: return [slope2b, interceptb]


            ax.plot(tau1[~np.isnan(slope)],interceptb+slope2b*tau1[~np.isnan(slope)],'green',linewidth=2)
            ax.set_title([r'$R^b$= ',str(r_valueb**2)+' b = '+ str(slope2b)])
            ax.set_xlabel(r'log $\tau_{1}$',fontsize=11)
            ax.set_ylabel(r'Learning rate b',fontsize=11)
            #legend(stateList,loc='center left', bbox_to_anchor=(1, 0.5),ncol=3)
            customaxis(ax,size=10,lw=2)
            #savefig('SSstatesAllData_v2.pdf', bbox_inches='tight' ,dpi=100)
            #show()
            #clf()
            """
            fig = figure(figsize=(6.,3))
            ax = subplot(1,1,1)
            listSlopes = (np.asarray(listSlopes))
            listSlopes2 = list(listSlopes)
            for i2 in listX:
                i3 = listSlopes2.pop(0)
                ax.scatter(i2,i3,s=20,facecolor =(0./256, 130./256, 130./256),edgecolor=(0./256, 130./256, 130./256),lw=0)
                #annotate(listStates.pop(0),(i2,i3+0.1))

            slope2, intercept, r_value, p_value, std_err = linregress(listX,listSlopes)
            print "slopes",slope2, intercept
            ax.plot(np.linspace(0,4,2),intercept+slope2*np.linspace(0,4,2),color=(0./256, 50./256, 130./256),linewidth=2)
            title([''.join([r'$R^2$= ',str(r_value**2)]) ,''.join(['-b= ',str(slope2)])])
            _L = []
            listX = np.asarray(listX)
            print listX
            print listSlopes
            print daysBetweenAttacks
            print np.log10(daysBetweenAttacks)
            t = np.linspace(min(listX),max(listX),6)
            x =  sorted(zip(listX,listSlopes))
            listX,listSlopes = zip(*x)

            t = np.linspace(0,len(x),10)
            _L1 = []
            _L2 = []
            for x in range(9):
                _L1.append(np.mean(listX[int(t[x]):int(t[x+1])]))
                _L2.append(np.mean(listSlopes[int(t[x]):int(t[x+1])]))

            #plot(_L1,_L2,'o',color='red',linewidth=2)


            xlines = [0,1,1.5,2,2.5,3,3.5,4]
            ylines = [0,0.5,1,1.5,2,2.5,3,3.5,4]#sorted([8,4,2,0,-2,-4,-10])

            """


        #customaxis(ax, c_right='none', c_top='none',lw=2, size=10, pad=8)
        #xlim([0,4])

        plt.xlim(1,4)
        plt.ylim(-3.5,3)
        savefig(fname+'SSInstRates_v2.pdf', bbox_inches='tight' ,dpi=100)
        show()

def fig1G():
    [slope2b,interceptb] = fig1EF(returnX = True)

    #1.83146071404 -5.08907637833 2.22318607529 -5.45669258405
    stateList = ["CA","AL","DC","FL","GA","IL","MI","NC","NY","OH","PA","SC","TN","TX","WA"]

    j = 0
    slope2 = slope2b
    intercept = interceptb
    bestErr = 9999
    out = -1
    allC = 51
    data = np.zeros([allC,allC])
    for slope2 in [slope2]:#linspace(-2.,8,allC):
        out += 1
        inL = -1
        for intercept in [intercept]:#linspace(-10,5,allC):
            inL += 1
            #108 0.308583133554 1.74832171778
            #slope2 = 1.1377
            #intercept = -2.5074
            j = 0
            #fig = figure(figsize=(1.5*12,1.5*10))
            ## Prediction
            totalAttacks= 0
            err0T = []#np.empty(0)
            err2Tx = []#np.empty(0)
            err1T = []#np.empty(0)
            err2T = []#np.empty(0)
            err3T = []#np.empty(0)
            err4T = []#np.empty(0)
            err5T = []#np.empty(0)
            err6T = []#np.empty(0)
            tempErr = []
            for state in stateList:
                j+=1
                [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2('ShootMiami2.csv',state=state)
                totalAttacks += len(daysBetweenAttacks)

                tau1 = np.log10(float(daysBetweenAttacks[0]))


                daysTheoret = 10**(tau1)*(np.arange(1.,len(ddd),dtype='float')**-((intercept+slope2*tau1)))
                #print state
                #print (daysTheoret)
                #print (daysBetweenAttacks)
                daysTheorAdj = np.zeros(len(daysBetweenAttacks))
                daysTheorAdjx = np.zeros(len(daysBetweenAttacks))
                daysTheorAd2 = np.zeros(len(daysBetweenAttacks))
                i = 0
                daysTheorAdj[0] = daysBetweenAttacks[0]
                daysTheorAdjx[0] = daysBetweenAttacks[0]
                daysTheorAd2[0] = daysBetweenAttacks[0]
                for times in daysBetweenAttacks[:-1]:
                    i+=1
                    daysTheorAdj[i] = float((times)*(((i+1.)/(i+2.))**-((intercept+slope2*tau1))))
                    #daysTheorAdj[i] = float((times))
                    #daysTheorAdjx[i] = float((times)
                    daysTheorAd2[i] = np.median(daysBetweenAttacks[np.max([0,i-3]):i])#float((times)*(2.**-((intercept+slope2*np.log10(times)))))



                #print daysTheorAdj
                #print daysTheorAd2
                daysTheoret2 = np.copy(daysTheoret)

                for i in range(len(daysBetweenAttacks)-1):
                    daysTheoret2[i+1] -= daysTheoret[i] - daysBetweenAttacks[i]


                daysTheorLinRegg = np.copy(daysBetweenAttacks)
                daysTheorLinReggSemiLog = np.copy(daysBetweenAttacks)
                daysTheorLinReggSemiLog2 = np.copy(daysBetweenAttacks)


                for i in range(len(daysBetweenAttacks)-2):
                    #linear regression,
                    slope2_t, intercept_t, r_value, p_value, std_err = linregress([1.,2.],daysBetweenAttacks[i:i+2])
                    daysTheorLinRegg[i+2] = intercept_t+slope2_t*3.

                    slope2_t, intercept_t, r_value, p_value, std_err = linregress(np.log10([1.,2.]),daysBetweenAttacks[i:i+2])
                    daysTheorLinReggSemiLog[i+2] = intercept_t+slope2_t*np.log10(3.)

                    slope2_t, intercept_t, r_value, p_value, std_err = linregress([1.,2.],np.log10(daysBetweenAttacks[i:i+2]))
                    daysTheorLinReggSemiLog2[i+2] = 10**(intercept_t+slope2_t*3.)

                err0 = (daysBetweenAttacks - daysTheoret2)/daysBetweenAttacks
                err2x = (daysBetweenAttacks - daysTheorAdjx)/daysBetweenAttacks
                err1 = (daysBetweenAttacks - daysTheoret)/daysBetweenAttacks


                err2 = (daysBetweenAttacks - daysTheorAdj)/daysBetweenAttacks

                err3 = (daysBetweenAttacks - daysTheorAd2)/daysBetweenAttacks
                err4 = (daysBetweenAttacks - daysTheorLinRegg)/daysBetweenAttacks
                err5 = (daysBetweenAttacks - daysTheorLinReggSemiLog)/daysBetweenAttacks
                err6 = (daysBetweenAttacks - daysTheorLinReggSemiLog2)/daysBetweenAttacks
                subplot(4,5,j)

                err0T.append(np.median(np.abs(err0[1:])))# = np.concatenate([err0T,np.abs(err0[1:])])
                err2Tx.append(np.median(np.abs(err2x[1:])))# = np.concatenate([err0T,np.abs(err0[1:])])
                #print "ererer", err0T,err0[1:]
                err1T.append(np.median(np.abs(err1[1:])))# = np.concatenate([err1T,np.abs(err1[1:])])
                err2T.append(np.median(np.abs(err2[1:])))# = np.concatenate([err2T,np.abs(err2[1:])])
                err3T.append(np.median(np.abs(err3[1:])))# = np.concatenate([err3T,np.abs(err3[1:])])
                err4T.append(np.median(np.abs(err4[2:])))# = np.concatenate([err4T,np.abs(err4[2:])])
                err5T.append(np.median(np.abs(err5[2:])))# = np.concatenate([err5T,np.abs(err5[2:])])
                err6T.append(np.median(np.abs(err6[2:])))# = np.concatenate([err6T,np.abs(err6[2:])])
                print state, np.round(err3[1:])


                hold(True)
                #plot(range(1,len(daysBetweenAttacks)),err0[1:],'o-',color='orange')
                #plot(range(1,len(daysBetweenAttacks)),err1[1:],'o-',color='red')
                #plot(range(1,len(daysBetweenAttacks)),err2[1:],'o-',color='blue')
                #plot(range(1,len(daysBetweenAttacks)),err3[1:],'o-',color='green')
                #plot(range(2,len(daysBetweenAttacks)),err4[2:],'o-',color='cyan')
                #plot(range(2,len(daysBetweenAttacks)),err5[2:],'o-',color='cyan')
                #plot(range(2,len(daysBetweenAttacks)),err6[2:],'o-',color='black')
                #plot(range(len(daysBetweenAttacks)),np.zeros(len(daysBetweenAttacks)))
                ylim([-2,2])
                #ylim([min(np.concatenate([err4,err5])),max(np.concatenate([err4,err5]))])
                title(state+str(np.median(np.abs(err3))))

                if state == stateList[-1]:
                    legend(['b) Same b, corrected','c) Same b, each attach','d) New b after each tau','f) Exponential interpolation'], loc='center left', bbox_to_anchor=(1, 0.5),ncol=1)
                    #red. Same b, if an attack happens before it should it's added to the next one
                    #blue. Same b, next attack calculated after each attack
                    #green. New b after each attack
                    #magenta. Linear regression in semilog plot (needs 3 points)

            data[out,inL] = int(np.median(err3T))
            #Predict
            if int(np.median(err3T)) < bestErr or bestErr == 9999:
                bestErr = int(np.median(err3T))
                bestS = slope2
                bestI = intercept
            print out, inL, bestErr, bestS, bestI

    print "aaa"
    print err2Tx
    print err2T
    data = data.T
    """
    fig, ax = plt.subplots()
    data[data>300] = 300
    imshow(data, interpolation='none')
    colorbar()
    ax.set_xticks(np.arange(data.shape[0])+0.5, minor=False)
    ax.set_yticks(np.arange(data.shape[1])+0.5, minor=False)
    #ax.invert_yaxis()
    ax.xaxis.tick_top()

    ax.set_xticklabels(linspace(-2,8,allC), minor=False,rotation=45)
    ax.set_yticklabels(linspace(-10,5,allC), minor=False)

    savefig('predictionBruteForce0.pdf', bbox_inches='tight' ,dpi=100)
    """
    err0T = 100*np.asarray(err0T)
    err1T = 100*np.asarray(err1T)
    err2T = 100*np.asarray(err2T)
    err2Tx = 100*np.asarray(err2Tx)
    err3T = 100*np.asarray(err3T)
    err5T = 100*np.asarray(err5T)
    err6T = 100*np.asarray(err6T)

    print bestErr, bestS, bestI
    tempErr = np.asarray(tempErr)
    print np.mean(tempErr), np.median(tempErr)
    suptitle('b =' + str(int(np.median(err0T)))+'. c =' + str(int(np.median(err1T)))+'. d =' + str(int(np.median(err3T)))+'. f =' + str(int(np.median(err5T))))
    print  (np.median(err0T)),(np.median(err1T)),(np.median(err2T)),(np.median(err3T)),(np.median(err4T)),(np.median(err5T)),(np.median(err6T))
    print  (np.mean(err0T)),(np.mean(err1T)),(np.mean(err2T)),(np.mean(err3T)),(np.mean(err4T)),(np.mean(err5T)),(np.mean(err6T))
    #plt.show()
    #savefig('predictionColl2.pdf', bbox_inches='tight' ,dpi=100)
    if int(np.median(err2T)) < bestErr:
        bestErr = int(np.median(err2T))
    show()
    clf()
    figure(figsize=(3,3))
    ax = subplot(1,1,1)
    import scipy
    x = err2T
    y = err3T
    print("Mann-Whitney U test.", scipy.stats.mannwhitneyu(x,y,use_continuity=True)[1]*2)
    print(scipy.stats.levene(x,y)[1])

    bar([0.1,0.5,0.9,1.4],[np.median(err2T),np.median(err3T),np.median(err5T),np.median(err6T)],0.35,
        yerr=[[0,0,0,0],[np.std(err2T),np.std(err3T),np.std(err5T),np.std(err6T)]],color=[(73./256, 142./256, 204./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256)],ecolor='k',edgecolor='none' )
    bar([2.0,2.4,2.8,3.2],[np.mean(err2T),np.mean(err3T),np.mean(err5T),np.mean(err6T)],0.35
        ,yerr=[[0,0,0,0],[np.std(err2T),np.std(err3T),np.std(err5T),np.std(err6T)]],color=[(73./256, 142./256, 204./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256)],ecolor='k',edgecolor='none' )
    #customaxis(ax,size=10,lw=2)
    ylabel('Percentage Error in Prediction',fontsize=11)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.tick_params(axis='both', direction='out')
    ax.get_xaxis().tick_bottom()   # remove unneeded ticks
    ax.get_yaxis().tick_left()

    tick_params(\
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off') # labels along the bottom edge are off

    ylim([0,1200])
    savefig('SSpredictionBarPlot_v2_median.pdf', bbox_inches='tight' ,dpi=100)
    show()




def intAttacks_v2(daysBetweenAttacks,plotB = 'False'):
    from scipy.stats import nanmedian
    num = 240

    def cosa(daysBetweenAttacks):
        ta = zeros(num)
        tb = zeros(num)
        for j in range(1,num+1):
            list1 = []
            list2 = []
            for i in range(len(daysBetweenAttacks)-1):
                if daysBetweenAttacks[i] <j:
                    list1.append(daysBetweenAttacks[i+1])
                else:
                    list2.append(daysBetweenAttacks[i+1])

            #print "Days between ", j
            #print len(list1),mean(list1),var(list1),median(list1)
            #print len(list2),mean(list2),var(list2),median(list2)
            if len(list2) > 3:
                ta[j-1] = mean(list2)
            else:
                ta[j-1] = np.NaN
            if len(list1) > 3:
                tb[j-1] = mean(list1)
            else:
                tb[j-1] = np.NaN

        return [ta,tb]


    if plotB == True:
        [ta,tb] = cosa(daysBetweenAttacks)
        fig = figure(figsize=(4.5,3))
        ax = subplot(1,1,1)
        hold(True)
        ax.plot(arange(1,num+1),ta,color=(73./256, 142./256, 204./256),linewidth=2)
        ax.plot(arange(1,num+1),tb,color='orange',linewidth=2)
        ax.plot(arange(1,num+1),zeros(len(ta))+mean(daysBetweenAttacks),'cyan')
        ylabel('Time to the next attack')
        #ylim([0,max(concatenate([ta,tb]))])
        #xlim([1,15])
        legend(['Mean of events without an attack in the previous i days','Mean of events with an attack in the previous i days'],3,fontsize=11,frameon=False)
        xlabel('i = Threshold (time with/without attack)',fontsize=11)
        customaxis(ax,size=10,lw=2)
        print (ta-tb), ta, tb
        xscale('log')

        tempAll = np.zeros(10000)
        tempAll2 = np.zeros(10000)
        for i in range(10000):

            tempDays = np.random.choice(daysBetweenAttacks,len(daysBetweenAttacks))
            [_1,_2] = cosa(tempDays)
            #ax.plot(arange(1,16),_1,color=(73./256, 142./256, 204./256),linewidth=2)
            #ax.plot(arange(1,16),_2,color='orange',linewidth=2)
            #ax.plot(arange(1,16),zeros(len(ta))+mean(tempDays),'cyan')

            print i, nanmedian(_1-_2)
            tempAll[i] = nanmedian(_1-_2)
            tempAll[i] = np.nanmean(_1-_2)
        print np.sum(tempAll>nanmedian(ta-tb))
        print np.sum(tempAll2>nanmean(ta-tb))

        title('p-values:'+str(np.sum(tempAll>nanmedian(ta-tb))/10000.)+' and '+ str(np.sum(tempAll2>nanmean(ta-tb))/10000. ))

        #show()#
        savefig('SSinteractionBetweenAttacks.pdf', bbox_inches='tight' ,dpi=100)
        #clf()
    return np.sum(np.abs(ta[:14]-tb[:14]))


def fig2A():
    [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2('ShootMiami2.csv')
    #[ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2('everytown.csv')
    #daysBetweenAttacks = daysBetweenAttacks[daysBet]
    intAttacks_v2(daysBetweenAttacks,True)



def fig2CD(numDays = 2, plotX=True):
    from mpl_toolkits.mplot3d import Axes3D
    [datesFinal,tweetsS,tweetsSS,tweetsAll,tweetsMS,tweetsMM] = studyTweets_v2()

    tweetsFinal = (tweetsMS+tweetsMM)/tweetsAll
    what = 'SS'
    fig = figure(figsize=(4.5,3))
    ax = fig.add_subplot(111)


    #fname = 'everytown.csv'
    #fname = 'ShootMiami2.csv'
    fname = "USAToday.csv"

    if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname)
    elif fname == "USAToday.csv": [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = studyUSAToday(fname)
    else:   [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2(fname)

    listPerc=[25,50,75]
    j = -1
    for threshold in percentile(daysBetweenAttacks,[25,50,75]):

        print j,threshold/np.median(daysBetweenAttacks)

        j+=1
        l1 = []
        l2 = []
        i = 0
        for timeD in datesFinal:
            temp = [(date - timeD).days for date in ddd]
            l2.append(tweetsFinal[i])
            if filter(lambda x: x > 0 and x <= threshold,temp):
                l1.append(1.)
            else:
                l1.append(0.)

            i+=1


        l1 = np.asarray(l1)
        l2 = np.asarray(l2)
        l1 = l1[~isnan(l2)]
        l2 = l2[~isnan(l2)]
        ind = np.nonzero(l1==1)[0][0]
        l1 = l1[ind:]
        l2 = l2[ind:]


        l1F = []
        l2F = []

        b = percentile(l2,range(0,100,1))
        inAnt = b[0]
        for inPos in b[:]:
            print("si",inPos)
            #l1F.append(np.nanmean(l1[np.nonzero(np.logical_and(l2>=inAnt,l2<inPos))[0]]))

            if len(np.nonzero(l2>inAnt)[0]) > 0:
                l1F.append(np.nanmean(l1[np.nonzero(l2>inAnt)[0]]))

                l2F.append((inAnt + inPos)/2.)
            inAnt = inPos

        l1 = np.asarray(l1F)
        l2 = np.asarray(l2F)



        if numDays > 1:
            slope1, intercept, r_value2, p_value, std_err = linregress(l2[:-1],l1[:-1])
        else:
            slope1, intercept, r_value2, p_value, std_err = linregress(l2[:],l1[:])



        if plotX:
            hold(True)
            print l2

            #ax.plot(l2[:-1],[threshold]*(len(l1)-1),l2[:-1]*slope1 + intercept,color='green',linewidth=3)
            #ax.scatter(l2,[threshold]*len(l1),l1,s=60,facecolor =(73./256, 142./256, 204./256),edgecolor=(73./256, 142./256, 204./256),lw=0)
            if what == "SS": ax.plot(l2[l2<100./1E6]*1E6,l1[l2<100./1E6],'-',color=((73)/256, (142.+30*j)/256, 204./256),linewidth=2,label='n = %i days' %threshold)
            else: ax.plot(l2*1E6,l1,'-',color=((73)/256, (142.+30*j)/256, 204./256),linewidth=2,label='n = %i days' %threshold)
            #title(tit+'. ' + 'R-value = ' +str(r_value2),fontsize=11)

            #show()
    plt.xscale('log')
    ax.set_ylabel('Probability of attack \nin the next n days',fontsize=12)
    ax.set_xlabel('Average tweets containing \n\\"mass\" and \"shooting\" per million',fontsize=12)
    ax.legend(frameon=False,fontsize=10)
    customaxis(ax,size=11,lw=2)
    xlim((np.min(l2*1E6),np.max(l2*1E6)))
    savefig(what+fname+'linear.pdf', bbox_inches='tight' ,dpi=100)
    show()
    """
    imshow(imMatrix.T,interpolation='None',cmap="BrBG",origin='lower', aspect='auto')
    plt.clim(0,1)
    colorbar()

    #ax.set_xticks(np.arange(data.shape[0])+0.5, minor=False)
    #ax.set_yticks(np.arange(data.shape[1])+0.5, minor=False)
    #ax.invert_yaxis()
    #ax.xaxis.tick_top()

    ax.set_yticklabels(range(100), l2, minor=False,rotation=45)
    ax.set_xticklabels(range(10), linspace(1./5,2.,10),minor=False)

    show()

    """
    return r_value2


def prediction():
    [slope2b,interceptb] = fig1EF(returnX = True)

    [datesFinal,tweetsS,tweetsSS,tweetsAll,tweetsMS,tweetsMM] = studyTweets_v2()

    tweetsS /= tweetsAll
    tweetsSS /= tweetsAll
    tweetsMS /= tweetsAll
    tweetsMM /= tweetsAll

    stateList = ["CA","AL","DC","FL","GA","IL","MI","NC","NY","OH","PA","SC","TN","TX","WA"]

    j = 0
    slope2 = slope2b
    intercept = interceptb
    totalAttacks= 0

    err2T = []#np.empty(0)

    X = [[],[],[],[],[],[]]
    Y = []
    for state in stateList:
        print state
        [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2('ShootMiami2.csv',state=state)
        #[ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2('everytown.csv',state=state)
        print len(ddd),len(daysBetweenAttacks)

        totalAttacks += len(daysBetweenAttacks)
        tau1 = np.log10(float(daysBetweenAttacks[0]))
        daysTheorAdj = np.zeros(len(daysBetweenAttacks))

        i = 0
        daysTheorAdj[0] = daysBetweenAttacks[0]
        twiiter_5_daysSS = []
        twiiter_5_daysS = []
        twiiter_5_daysMS = []
        twiiter_5_daysMM = []
        daysBA2 = []
        daysBA = []
        daysTh = []
        pTime = daysBetweenAttacks[0]
        for times in daysBetweenAttacks[1:]:
            j+=1
            i+=1
            # i+1 because the daysBetweenAttacks refer
            temp = np.asarray([(date - ddd[i+1]).days for date in datesFinal])
            index = np.nonzero(np.logical_and(temp>-5,temp<=0))[0]
            print index
            if len(index) > 0:
                ind = index
                twiiter_5_daysSS += [np.max(tweetsSS[ind][0:5])*1E7]
                twiiter_5_daysS += [np.max(tweetsS[ind][0:5])*1E6]
                twiiter_5_daysMS += [np.max(tweetsMS[ind][0:5])*1E7]
                twiiter_5_daysMM += [np.max(tweetsMM[ind][0:5])*1E7]

                daysBA2.append(pTime)
                daysBA.append(times)
                b = (intercept + slope2*np.log10(pTime))
                print b
                nTime = pTime*2**(-b)

                daysTh.append(float((pTime)*(2.**-((intercept+slope2*tau1)))))

            daysTheorAdj[i] = float((pTime)*(2.**-((intercept+slope2*tau1))))
            pTime = times
        plt.clf()

        plot(daysTh,label='Th')
        plot(daysBA,label='Y')
        plot(daysBA2,label='X')
        plt.legend()
        #show()

        err2 = (daysBetweenAttacks - daysTheorAdj)/daysBetweenAttacks
        err2T.append(np.median(np.abs(err2[1:])))# = np.concatenate([err2T,np.abs(err2[1:])])

        X[0] += list(daysBA2)
        X[1] += list(daysTh)


        X[2] += list(twiiter_5_daysSS)
        X[3] += list(twiiter_5_daysS)
        X[4] += list(twiiter_5_daysMS)
        X[5] += list(twiiter_5_daysMM)


        Y += list(daysBA)



    X =  np.asarray(X).T
    Y = np.asarray(Y)

    print np.shape(X)
    print j
    #bar([0.1,0.5,0.9],[np.median(err2T),np.median(err5T),np.median(err6T)],0.35,yerr=[[0,0,0],[np.std(err2T),np.std(err5T),np.std(err6T)]],color=[(73./256, 142./256, 204./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256)],ecolor='k',edgecolor='none' )
    #bar([1.6,2.0,2.4],[np.mean(err2T),np.mean(err5T),np.mean(err6T)],0.35,yerr=[[0,0,0],[np.std(err2T),np.std(err5T),np.std(err6T)]],color=[(73./256, 142./256, 204./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256)],ecolor='k',edgecolor='none' )
    from sklearn import datasets, svm,cross_validation, linear_model

    a_train, a_test, b_train, b_test = cross_validation.train_test_split(X[:,:], Y, test_size=0.1)#, random_state=42)
    print np.shape(a_train)
    plt.clf()
    for i in range(6):
        plot(a_train[:,i])
    plot(a_test)
    legend(['pT','pred','SS','S','MS','MM','True'])
    #show()

    clf = linear_model.LinearRegression(normalize=True)

    # Train the model using the training sets
    clf.fit(X, Y)

    # The coefficients
    print('Coefficients: \n', clf.coef_)
    # The mean square error
    print("Residual sum of squares: %.2f"
          % np.mean((clf.predict(a_test) - b_test) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % clf.score(a_test, b_test))

    #clf = svm.SVC(kernel='rbf')
    #clf.fit(a_train, b_train)
    #Z = clf.predict(a_test)
    print np.shape(a_test)
    print clf.predict(a_test)

    print b_test

    plt.clf()
    for i in range(6):
        plot(a_test[:,i])
    plot(b_test)
    plot(clf.predict(a_test))
    legend(['pT','pred','SS','S','MS','MM','True','Pred'])
    #show()

    print np.median(err2T)
    print np.median(np.abs(clf.predict(a_test)-b_test)/b_test)
    print np.median(np.abs(a_test[:,1]-b_test)/b_test)

    print np.mean(err2T)
    print np.mean(np.abs(clf.predict(a_test)-b_test)/b_test)
    print np.mean(np.abs(a_test[:,1]-b_test)/b_test)
    err2T = []
    err3T = []
    err4T = []
    counter = 0
    for state in stateList:
        [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2('ShootMiami2.csv',state=state)
        print len(ddd),len(daysBetweenAttacks)

        totalAttacks += len(daysBetweenAttacks)
        tau1 = np.log10(float(daysBetweenAttacks[0]))
        daysTheorAdj = np.zeros(len(daysBetweenAttacks))

        i = 0
        daysTheorAdj[0] = daysBetweenAttacks[0]
        twiiter_5_daysSS = []
        twiiter_5_daysS = []
        twiiter_5_daysMS = []
        twiiter_5_daysMM = []
        daysBA2 = []
        daysBA = []
        daysTh = []
        pTime = daysBetweenAttacks[0]
        cosa = []

        for times in daysBetweenAttacks[1:]:
            j+=1
            i+=1
            # i+1 because the daysBetweenAttacks refer
            temp = np.asarray([(date - ddd[i+1]).days for date in datesFinal])
            index = np.nonzero(np.logical_and(temp>-5,temp<=0))[0]
            print index
            if len(index) > 0:
                    ind = index
                    twiiter_5_daysSS += [np.max(tweetsSS[ind][0:5])*1E7]
                    twiiter_5_daysS += [np.max(tweetsS[ind][0:5])*1E6]
                    twiiter_5_daysMS += [np.max(tweetsMS[ind][0:5])*1E7]
                    twiiter_5_daysMM += [np.max(tweetsMM[ind][0:5])*1E7]

                    daysBA2.append(pTime)
                    daysBA.append(times)
                    b = (intercept + slope2*np.log10(pTime))
                    print b
                    nTime = pTime*2**(-b)

                    daysTh.append(float((pTime)*(2.**-((intercept+slope2*tau1)))))

            daysTheorAdj[i] = float((pTime)*(2.**-((intercept+slope2*tau1))))
            pTime = times
        cosa = np.abs(np.asarray(daysBA)-clf.predict(np.array([daysBA2,daysTh,twiiter_5_daysSS,twiiter_5_daysS,twiiter_5_daysMS,twiiter_5_daysMM]).T))/np.asarray(daysBA)
        cosa2 = np.abs(np.asarray(daysBA)-np.asarray(daysTh))/np.asarray(daysBA)
        print "cosa", cosa
        err2 = (daysBetweenAttacks - daysTheorAdj)/daysBetweenAttacks
        err2T.append(np.median(np.abs(err2[1:])))# = np.concatenate([err2T,np.abs(err2[1:])])

        if len(cosa)>0:
            counter += 1
            print state,counter
            print daysBA
            print clf.predict(np.array([daysBA2,daysTh,twiiter_5_daysSS,twiiter_5_daysS,twiiter_5_daysMS,twiiter_5_daysMM]).T)
            err3T.append(np.median(cosa))# = np.concatenate([err2T,np.abs(err2[1:])])
            err4T.append(np.median(cosa2))# = np.concatenate([err2T,np.abs(err2[1:])])

    sleep()
    print err2T
    print err3T
    print err4T
    print err3T[err3T<20]
    err3T = np.asarray(err3T)
    print np.median(err2T)
    print err3T
    print np.median(err3T[err3T<20])
    print np.median(err4T)
    print np.std(err2T)
    print np.std(err3T[err3T<20])
    print np.std(err4T)
    print np.mean(err2T)
    print np.nanmean(err3T[err3T<20])
    print np.nanmean(err4T)
    plt.clf()
    figure(figsize=(3,3))
    ax = subplot(1,1,1)

    bar([0.1,0.5,0.9],[np.median(err2T),np.median(err3T[err3T<20]),np.median(err4T)],0.35,yerr=[[0,0,0],[np.std(err2T),np.std(err3T[err3T<20]),np.std(err4T)]],color=[(73./256, 142./256, 204./256),(241./256, 98./256, 67./256),(241./256, 98./256, 67./256)],ecolor='k',edgecolor='none' )
    show()

def marginalProbability():
    import statsmodels.api as sm
    for fname in ['ShootMiami2.csv']:
        for name in [0]:#'NCollege','College',0]:
            if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname,type=name)
            else: [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2(fname,type=name)
            h = 0
            for nbins in range(4,11,1):
                h += 1
                bins =  np.sort(daysBetweenAttacks)[np.asarray([int(_) for _ in np.linspace(0,len(daysBetweenAttacks)-1,nbins)])]

                logDays = (daysBetweenAttacks)

                freqM,_,_ = np.histogram2d(logDays[:-1],logDays[1:],bins= bins)
                freqBoth,bins = np.histogram(logDays,bins = bins)
                #freqBoth = freqBoth/np.sum(freqBoth)
                #freqM = freqM/np.sum(freqM)
                print bins
                m1 = np.sum(freqM)

                marginal = []
                both = []
                for i in range(len(bins)-1):
                    for j in range(len(bins)-1):

                        freqM[i,j] /= (float(freqBoth[i]*freqBoth[j])/np.sum(freqBoth)**2)*m1
                        #marginal.append(freqM[i,j])
                        #both.append((freqBoth[i]*freqBoth[j]))

                #plt.plot(both,marginal,'o')
                #lowess = sm.nonparametric.lowess
                #a = lowess(marginal,both,frac = 0.66,delta=0.0)
                #plt.plot(a[:,0],a[:,1], linewidth=2,color='green')
                plt.subplot(3,3,h)
                plt.imshow(freqM,interpolation='none',origin='lower')
                plt.colorbar()
        plt.show()

def marginalProbability2():
    import statsmodels.api as sm
    for fname in ['ShootMiami2.csv']:#['everytown.csv']:#
        for name in [0]:#'NCollege','College',0]:
            if fname == 'ShootMiami2.csv': [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname,type=name)
            elif fname == "USAToday.csv": [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = studyUSAToday(fname,type=name)
            else: [ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysEveryTown_v2(fname,type=name)
            fig = figure(figsize=(4.5,3.))
            ax =  fig.add_subplot(1,1,1)
            h = 0
            for nbins in [7]:
                h += 1
                bins =  np.sort(daysBetweenAttacks)[np.asarray([int(_) for _ in np.linspace(0,len(daysBetweenAttacks)-1,nbins)])]
                bins = [int(_) for _ in bins]
                print bins

                x = []
                y = []
                for i in range(len(daysBetweenAttacks)-1):
                    if daysBetweenAttacks[i] < 7 :
                        x.append(daysBetweenAttacks[i+1])
                    else:
                        y.append(daysBetweenAttacks[i+1])
                x = np.asfarray(x)
                y = np.asfarray(y)

                freqX,bins= np.histogram(x,bins=bins)
                freqY,bins= np.histogram(y,bins=bins)
                freqX = np.asfarray(freqX)/np.sum(freqX)
                freqY = np.asfarray(freqY)/np.sum(freqY)
                t = np.vstack([freqX,freqY])

                plt.imshow(t,interpolation='none',origin='lower',cmap='Blues')
                plt.colorbar(orientation='horizontal')
                plt.xlabel('Time to next attack (Days)',fontsize=10)
                plt.xticks(range(len(bins)-1),[str(_)+" - " + str(_2) for (_,_2) in zip(bins[:-1],bins[1:])],fontsize=10)
                plt.yticks([0,1],["Attack in \n previous 7 days", "No Attack in \n previous 7 days"],fontsize=10)

            savefig('fig1B.pdf', bbox_inches='tight' ,dpi=10)
            savefig('fig1B.png', bbox_inches='tight' ,dpi=10)
            plt.show()



def main1():
    #studyTweets_v2()
    #testX()

    fig1A() ##Pooewr-law
    #fig1BCD() ## lowess. this
    #fig1EF() # Regression
    #fig1G() #Prediction


    #residualsMiami_v2()
    #marginalProbability()
    #marginalProbability2() #. this

    #fig2A()
    #fig2CD()

    #residualsMiami_v2()
    #prediction()
    #fname = 'ShootMiami2.csv'
    #[ddd,sizeEvent,sizeEvent2,daysBetweenAttacks,cV,cV2,vtm] = regetDataDaysMiami_v2(fname)
    #plot(daysBetweenAttacks,sizeEvent2[1:],'o')
    #show()


if __name__ == '__main__':
     main1()
